In [1]:
!pip install -U openai==0.28.0
!pip install -U langchain

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain.llms import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [3]:
summary_template = """你是一位非常聪明的中文文本摘要专家。
你擅长以简洁易懂的方式把原文中文文本提取成为中文文本摘要。

这是一个原文：
{input}"""


translation_template = """你是一位很棒的英文翻译专家。你擅长把原文英文翻译成中文。

这是一个原文：
{input}"""

In [4]:
prompt_infos = [
    {
        "name": "文本摘要",
        "description": "适用于原文中文文本摘要",
        "prompt_template": summary_template,
    },
    {
        "name": "翻译",
        "description": "适用于原文英文翻译成中文",
        "prompt_template": translation_template,
    },
]

In [5]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [6]:
llm = OpenAI()

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.llms.openai.OpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAI`.
  warn_deprecated(


In [7]:
destination_chains = {}

for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

default_chain = ConversationChain(llm=llm, output_key="text")

In [8]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [9]:
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [10]:
print(destinations)

['文本摘要: 适用于原文中文文本摘要', '翻译: 适用于原文英文翻译成中文']


In [11]:
print(destinations_str)

文本摘要: 适用于原文中文文本摘要
翻译: 适用于原文英文翻译成中文


In [12]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
文本摘要: 适用于原文中文文本摘要
翻译: 适用于原文英文翻译成中文

<< INPUT >>
{input}

<< OUTPUT (must include ```json at the start

In [13]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [14]:
print(chain.run("他是个独自在湾流中一条小船上钓鱼的老人，至今已去了八十四天，一条鱼也没逮住。头四十天里，有个男孩子跟他在一起。可是，过了四十天还没捉到一条鱼，孩子的父母对他说，老人如今准是十足地倒了血霉，这就是说，倒霉到了极点，于是孩子听从了他们的吩咐，上了另外一条船，头一个礼拜就捕到了三条好鱼。孩子看见老人每天回来时船总是空的，感到很难受，他总是走下岸去，帮老人拿卷起的钓索，或者鱼钩和鱼叉，还有绕在桅杆上的帆。帆上用面粉袋片打了些补丁，收拢后看来象是一面标志着永远失败的旗子。"))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(




> Entering new MultiPromptChain chain...
文本摘要: {'input': '他是个独自在湾流中一条小船上钓鱼的老人，至今已去了八十四天，一条鱼也没逮住。头四十天里，有个男孩子跟他在一起。可是，过了四十天还没捉到一条鱼，孩子的父母对他说，老人如今准是十足地倒了血霉，这就是说，倒霉到了极点，于是孩子听从了他们的吩咐，上了另外一条船，头一个礼拜就捕到了三条好鱼。孩子看见老人每天回来时船总是空的，感到很难受，他总是走下岸去，帮老人拿卷起的钓索，或者'}
> Finished chain.
拿他那副沉重的桨。老人总是很感激这种帮助，但他总是知道这样做是徒劳无功的。

他是一位老人，独自驾驶小船在湾流中钓鱼，已经八十四天了，却一无所获。前四十天，有一个男孩陪伴他。但是，当四十天过去了还没有捕获一条鱼时，孩子的父母认为老人的运气已经彻底倒霉，于是孩子听从父母的建议，换了一条船，第一个星期就捕获了三条好鱼。孩子看到老人每天回来都是空船，十分难过，总是走下岸帮忙拿钓索或者划那副沉重的桨。


In [15]:
print(chain.run("Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans. Last year COVID-19 kept us apart. This year we are finally together again. Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. With a duty to one another to the American people to the Constitution. And with an unwavering resolve that freedom will always triumph over tyranny."))



> Entering new MultiPromptChain chain...


/usr/local/lib/python3.10/dist-packages/langchain/chains/llm.py:316: UserWarning: The predict_and_parse method is deprecated, instead pass an output parser directly to LLMChain.
  warnings.warn(


翻译: {'input': 'Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans. Last year COVID-19 kept us apart. This year we are finally together again. Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. With a duty to one another to the American people to the Constitution. And with an unwavering resolve that freedom will always triumph over tyranny.'}
> Finished chain.

议长女士，副总统女士，我们的第一夫人和第二绅士。国会议员和内阁成员。最高法院法官们。我的美国同胞们。去年，新冠疫情让我们分隔两地。今年，我们终于再次聚集在一起。今晚，我们作为民主党人、共和党人和独立人士相遇。但最重要的是，作为美国人。我们有责任相互关怀，为美国人民和宪法服务。并坚定不移地相信，自由将永远战胜暴政。
